In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [8]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [10]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)


# Exploratory Precipitation Analysis

In [11]:

# Using the inspector to print the column names for Measurement
measurement_columns = inspector.get_columns('measurement')
print("Measurement Table")
for column in measurement_columns:
    print(column['name'], column["type"])



Measurement Table
id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [12]:
# Using the inspector to print the column names for Station
station_columns = inspector.get_columns('station')
print("Station Table")
for column in station_columns:
    print(column['name'], column["type"])

Station Table
id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [13]:
# Find the most recent date in the data set.
recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
print(f"Most recent date is: {recent_date[0]}")

Most recent date is: 2017-08-23


In [15]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

new_rec_date = dt.datetime.strptime(recent_date[0], '%Y-%m-%d')
new_rec_date.strftime('%Y-%m-%d')


# Calculate the date one year from the last date in data set.

one_year_date = (new_rec_date - dt.timedelta(days=365)).strftime('%Y-%m-%d')
one_year_date

# Perform a query to retrieve the data and precipitation scores
prcp_query_year = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_year_date).\
                                                                    filter(Measurement.date <= new_rec_date).all()
prcp_query_year

# Save the query results as a Pandas DataFrame and set the index to the date column
prcp_scores_df = pd.DataFrame(prcp_query_year, columns = ["date", "prcp"]).set_index("date")
prcp_scores_df.head()

# Sort the dataframe by date

prcp_scores_df.sort_values(by=['date'])
prcp_scores_df.head()

# Use Pandas Plotting with Matplotlib to plot the data




,prcp
date,
2016-08-23,0.00
2016-08-24,0.08
2016-08-25,0.08
2016-08-26,0.00
2016-08-27,0.00


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()